In [ ]:
!pip install datasets

In [ ]:
!pip install lingfeat

In [ ]:
import numpy as np
import pandas as pd
from lingfeat import extractor
from datasets import load_dataset

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
dataset = load_dataset("onestop_english")

In [ ]:
dataset = dataset.rename(columns={'text':'content', 'target':'labels'})

Mean sentence length

In [ ]:
sen_len = []
for i in dataset['content']:
  curr_sen_len = []
  for j in sent_tokenize(i):
    curr_sen_len.append(len(word_tokenize(j)))
  mean_len = np.mean(curr_sen_len)
  sen_len.append(mean_len)

In [ ]:
dataset['mean_sentence_len'] = sen_len

Mean dependency tree height

In [ ]:
en_nlp = spacy.load('en_core_web_sm')

In [ ]:
def walk_tree(node, depth):
    if node.n_lefts + node.n_rights > 0:
        return max(walk_tree(child, depth + 1) for child in node.children)
    else:
        return depth

def get_mean_tree_height(text):
  depths = []
  sentences = sent_tokenize(text)
  for sentence in sentences:
    doc = en_nlp(sentence)
    depth = [walk_tree(sent.root, 0) for sent in doc.sents]
    depths.extend(depth)
  return np.mean(depths)

In [ ]:
dataset['mean_tree_height'] = dataset['content'].progress_apply(get_mean_tree_height)

Average count of Noun Phrases per sentence

In [ ]:
def get_avg_nounphrase(text):
  LingFeat = extractor.pass_text(text)
  LingFeat.preprocess()
  avg = LingFeat.PhrF_()['as_NoPhr_C']
  return avg

In [ ]:
dataset['avg_nounphrases'] = dataset['content'].progress_apply(get_avg_nounphrase)